# Dialog (formatting, tokenization,...)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google-deepmind/dialog/blob/main/dialog/colab/dialog.ipynb)

Dialog is a mini library to simplify creating, manipulating and visualizing model examples, including:

* Formatting
* Tokenization
* System instructions, thinking
* Multi-turns
* Multimodal
* Tool use
* Visualization


In [ ]:
# !pip install -q dialog

In [46]:
from etils import ecolab

with ecolab.adhoc():
  import dialog

# Basics

Here's a simple example on creating a conversation:

In [7]:
conv = dialog.Conversation(
    dialog.User('Hello'),
    dialog.Model('Hello, what can I do for you ?'),
    dialog.User('Can you tell me an uplifting story ?'),
)
conv

The conversation can be exported as text with `.as_text()`.

2 modes exists:

* Inference (default): The exported text is the prompt fetch to prefill the model. It always ends by an incomplete model turn (i.e. an unclosed `<|turn>model\n`). An empty start of model turn is automatically added.

In [8]:
conv.as_text()

* Training: The conversation is assumed to be the full complete example. Always ends by `<turn|>`

In [9]:
conv.as_text(training=True)

Or exported into model token inputs with `.as_numpy()`. This returns a `DialogArray` dataclass with `.text`, `.image`, `.audio` fields.

In [ ]:
conv.as_numpy()  # TODO(epot): !!!

## Manipulation

Conversations objects are **immutable**, but can be manipulated easily.

Turns/chunks can be be added using `+` operator  (For example, to add a system instruction):

In [10]:
conv = dialog.System('User name: Alexandre\nLocation: Paris') + conv

conv

Standard slicing, iteration, len,... operations are supported:

In [11]:
assert len(conv[:2]) == 2  # Get the sub-conversation

conv[-1]  # Get an individual turn

Text can be automatically parsed back. This is especially helpful to parse tool calls generated by the model (Note that the last empty model turn (`<|turn>model\n`) get filtered):

In [14]:
conv = dialog.Conversation("""
<|turn>system
Current location: Paris

<|tool>declaration:get_weather{description:<|"|>Get current weather information for a location<|"|>,parameters:{type:<|"|>object<|"|>,properties:{location:{type:<|"|>string<|"|>,description:<|"|>City name or zip code<|"|>}}}}<tool|><turn|>
<|turn>user
What should we work on today ?<turn|>
<|turn>model
""")

conv

# Advanced

## Thinking

* To activate thinking, add the special `dialog.DeepThinkOn()` token in the system instruction.
* Model thoughts can be added with `dialog.Thought(...)`

In [18]:
conv = dialog.Conversation(
    dialog.System(
        dialog.Think(),
        '\n\nAsk follow up questions.',  # If adding other instructions, prefix them with `\n\n`
    ),
    dialog.User('Hello'),
    dialog.Model(
        dialog.Thought('User is greeting me. I should greet him too.'),
        'Hello, what can I do for you ?',
    ),
)
conv

Thinking is formatted as:

1. The `<|think|>` special token is added in the system instruction.
2. The model thoughts are formatted inside the `<|channel>thought` tag.

In [19]:
conv.as_text()

IMPORTANT: In multi-turn conversations, before feeding the conversations to the model, the previous model thoughts should be removed. This can be done with `.strip_thoughts()`

If the conversation ends by a model turn, only thoughts from previous turns will be stripped. This is because conversations ending by model turn represent incomplete generations (for model with tool calls).

In [20]:
conv = dialog.Conversation(
    dialog.System(dialog.Think()),
    dialog.User('2+2='),
    dialog.Model(dialog.Thought('This is simple math.'), '4'),
    dialog.User('2*2='),
    dialog.Model(dialog.Thought('Also easy.'), '4'),
    dialog.User('2-2='),
    dialog.Model(dialog.Thought('Let\'s keep going.'), '0'),
)

# Only the last turn thoughts are preserved
conv.strip_thoughts()

## Multimodality

Users, system turns can include multi-modal chunks (`dialog.Image`, `dialog.Audio`).

Data can be provided as:

* `https://example.org/img.jpg`: Url
* `/path/to/img.jpg`: Path (or any pathlib-like object)

Additionally, the following types are supported:

For images:

* `np.array`: uint8 3d numpy array.
* `PIL.Image.Image`

For audio:

* `pydub.AudioSegment`

In [4]:
conv = dialog.Conversation(
    dialog.User(
        'Describe this image: ',
        dialog.Image('https://www.gstatic.com/aistudio/sample-images/animal1.jpg'),
        '\nAnd this audio: ',
        dialog.Audio('https://www.gstatic.com/play_books_pfe/audiosamples/gemma.mp3'),
    )
)

conv

When exported, `.as_text()` will use a single token placeholder, which won't be correctly tokenizer. This should only be used for debugging/inspection purposes.

In [ ]:
conv.as_text()

Instead, `.as_numpy()` should be used, which will tokenize the image as `[..., '<|image>', placeholder, placheholder, ... placeholder, '<image|>', ...]`. `placeholder` are then later-on replaced inside the model by the actual image tokens.

## Function calling

You can define tools as:

* Json dict
* `mcp.Mcp()`


In [47]:
import mcp

tool_1 = mcp.Tool(
    name='get_weather',
    description="Get current weather information for a location",
    inputSchema={
    "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City name or zip code"
            },
            "timedelta": {
                "type": "integer",
                "description": "Number of days in the future. Default to 0.",
            },
        },
    },
)
tool_2 = {
    'name': 'get_location',
    'description': 'Get the user current location',
    'inputSchema': {},
}

Available tools should be added in the System instructions.

In [59]:
conv = dialog.Conversation(
    dialog.System(
        dialog.Tool(tool_1),
        dialog.Tool(tool_2),
    ),
    dialog.User('Where are we ?'),
)
conv

The model will generate one or more `<|tool_call><tool_call|>`. Sampling will stop when the model generate the `<|tool_response>` token.

The model output string can then be parsed back and the tool call and result can then be fetch back to the model to generate the final answer.

In [60]:
conv += dialog.Model(
    # Tool call generated by the model
    dialog.ToolCall('<|tool_call>call:get_location{}<tool_call|>'),
    # Function response fed back to the model
    dialog.ToolResponse({
        'name': 'get_location',
        'structuredContent': {'result': 'Paris, France'},
    }),
    # Final model answer
    'We are currently in Paris.',
)
conv

Note: Contrary to MCP specifications, the `ToolResponse` require:
* The `name` of the function
* The `structuredContent` (and not `content`)


## Pro-tip: The magic ;

When using `from etils import ecolab`, the conversations can be auto-displayed anywhere using a trailing `;` (including in assignment `x = y;`):

In [67]:
conv = dialog.Conversation(
    dialog.User('Hello'),
    dialog.Model('Hello'),
);  # Display the full conversation

for turn in conv:
  turn + ' world!';  # Display individual turns

This avoid boilerplate `IPython.display.display()`. See [documentation](https://etils.readthedocs.io/en/latest/ecolab.html#pretty-display-with-trailing).

## Next step

See the sampling tutorial to use `dialog` with actual models.